<a href="https://colab.research.google.com/github/shubhamksingh1/RecommendationEngine/blob/main/CollaborativeFiltering/Perspective/Test_Collaborative_Recommendation_Perspective_Keras_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pickle
import pandas as pd
from keras.models import load_model 
import numpy as np

# KNN MATRIX FACTORIZATION

## Read Files

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/rating_without_zero_v1.pkl")

model_knn = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/model_knn.pkl", "rb" ) )

df_matrix = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/uid_asset_mat.pkl", "rb" ) )
df2 = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/user_mean_rating.pkl")

inddict = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/inddict.pkl", "rb" ) )

ds = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/recommendation_v4.pkl")

## Function Definition

In [ ]:
def find_similar_users(index):
    distances, indices = model_knn.kneighbors(df_matrix.iloc[index, :].values.reshape(1, -1), n_neighbors = 21)
    similarities = 1 - distances.flatten() #Converting to 1-Dimention and increasing score for nearer users
    
    return similarities, indices

In [ ]:
def predict_rating(user_id, asset_id):
    unique_users = list(set(df['uid'].values))
    user_index = unique_users.index(user_id)
    unique_assets = list(set(df['asset_id'].values))
    asset_index = unique_assets.index(asset_id)
    similarities, indices = find_similar_users(user_index)
    mean_rating = df2['Mean_Rating'][user_index]
    #mean_rating = df_matrix.iloc[user_index,:].mean()
    sum_wt = np.sum(similarities) - 1
    weighted_sum = 0
    prediction = 0
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_index:
            continue;
        elif (df_matrix.iloc[indices.flatten()[i],asset_index]) != 0:
            j = indices.flatten()[i] 
            ratings_diff = df_matrix.iloc[j, asset_index] - df2['Mean_Rating'][j]#np.mean(df_matrix.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            weighted_sum = weighted_sum + product
    prediction = mean_rating + (weighted_sum/sum_wt)
    
    return prediction

In [ ]:
def get_recommendations(user_id, n):
    n = n + 1
    assets = []
    ratings = []
    unique_assets = list(set(df['asset_id'].values))
    asset_rated_by_user = df.loc[df['uid'] == user_id, 'asset_id'] # Assets of which user 'user_id' already rated
    if len(asset_rated_by_user) >= 3:
        assets_to_pred = np.setdiff1d(unique_assets, asset_rated_by_user) # Remove assets that 'user_id' has rated from list of all assets
        for i in assets_to_pred:
            asset_rating = predict_rating(user_id, i) 
            ratings.append(asset_rating)
        pred_ratings = np.array(ratings)
        indices = pred_ratings.argsort()[:-n:-1]
        for i in indices:
            assets.append(assets_to_pred[i])
        iids = [inddict[asset_id] for asset_id in assets]
        ds2 = ds.loc[:, ['asset_id', 'asset_title', 'asset_description', 'Domain']]
        return ds2.iloc[iids]
    else:
        return 'User has rated less than 3 articles'
    

## Get Recommendations

Users      Domain
*   1 - 12 :CX
*   13 - 24 : Agile
*   25 - 36 : AI
*   37 - 48 : IOT
*   49 - 60 : BigData
*   61 - 72 : Cloud
*   73 - 84 : HR
*   85 - 100 : Industry

In [ ]:
ds.Domain.value_counts()

Industry    21
CX          19
Agile       10
AI           8
Cloud        7
BigData      6
IOT          6
HR           5
Name: Domain, dtype: int64

In [ ]:
get_recommendations(1, 5)

,asset_id,asset_title,asset_description,Domain
0,1,the ascent of the smarter faster lighter enter...,peter drucker the legendary management scholar...,CX
13,14,when cx should begin with tweaks not disruption,today s most over used corporate cliché is dis...,CX
71,74,proactive protection how companies can secure ...,today digital customer experience transformati...,CX
40,41,the magic that happens when manufacturers digi...,a growing number of manufacturers are digitall...,IOT
39,40,now you can simulate nearly anything,organizations have long performed what if simu...,Cloud


In [ ]:
get_recommendations(14, 5)

,asset_id,asset_title,asset_description,Domain
62,65,an agile manifesto for the board,it s hard to avoid the term agile these days t...,Agile
1,2,agile s day has truly arrived,the software development community has been us...,Agile
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
25,26,training for the m a marathon,a few years ago two airlines announced a merge...,Industry
19,20,reimagining the contact center with digital te...,it costs billion per year to field the billion...,CX


In [ ]:
get_recommendations(28, 5)

,asset_id,asset_title,asset_description,Domain
57,59,taking a machine first approach to digital tra...,many companies believe they can compete in the...,AI
66,69,how intelligent automation is rewriting the ru...,customer support has become a key differentiat...,AI
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
21,22,why your cx must soon plug and play into a dig...,digital ecosystems increasingly are coalescing...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(47,5)

,asset_id,asset_title,asset_description,Domain
3,4,using iot data to understand how your products...,we have all seen some eye bulging numbers in r...,IOT
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(55,5)

,asset_id,asset_title,asset_description,Domain
54,55,digital data s greatest opportunities,every company today must digitally transform i...,BigData
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
24,25,how the ceo can propel a cx initiative,whether it s a series of tweaks or a wholesale...,CX


In [ ]:
get_recommendations(63,5)

,asset_id,asset_title,asset_description,Domain
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
21,22,why your cx must soon plug and play into a dig...,digital ecosystems increasingly are coalescing...,CX
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX


In [ ]:
get_recommendations(74,5)

,asset_id,asset_title,asset_description,Domain
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
30,31,how to make location independent agile work,while it is very beneficial to have all the te...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud
24,25,how the ceo can propel a cx initiative,whether it s a series of tweaks or a wholesale...,CX


In [ ]:
get_recommendations(80,5)

,asset_id,asset_title,asset_description,Domain
81,85,why the emotional wellbeing of your employees ...,abstract as nations around the world deal with...,HR
28,29,students for life why executives must continua...,companies today must get smarter faster and li...,HR
29,30,embracing agility means agility by the busines...,the push for agility in software development s...,Agile
22,23,what comes after cxm helping customers help ot...,thirty years ago large global companies began ...,CX
23,24,the sure but winding road to the cloud,even for companies that can afford billions of...,Cloud


In [ ]:
get_recommendations(83,5)

'User has rated less than 3 articles'

In [ ]:
get_recommendations(95,5)

,asset_id,asset_title,asset_description,Domain
64,67,unleashing business value in a business world,in an era we call business digital technologie...,Industry
55,56,how the digital renovators are racing ahead,numerous studies show that the majority of lar...,Industry
79,83,tcs cmo study stage conversion personalizing c...,chief marketing officers have one of the riski...,Industry
38,39,why your products must be smart and connected,it s become increasingly conceivable and cost ...,Industry
51,52,from making autos to making software the evolu...,michael a cusumano1 is a professor at the mass...,Industry


# Keras Model

## Read Saved Files and Model

In [ ]:
model = load_model('/content/drive/MyDrive/Collaborative_Recommendation/Keras/Model/modelv1')
asset_df = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/MatrixFactorization/recommendation_v4.pkl")
df_keras_ratings = pd.read_pickle("/content/drive/MyDrive/Collaborative_Recommendation/Keras/ratingsv1keras.pkl")


In [ ]:
asset2asset_encoded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/asset2asset_encoded.pkl", "rb" )) 
user2user_encoded = pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/user2user_encoded.pkl", "rb" ) )    
asset_encoded2asset= pickle.load(open("/content/drive/MyDrive/Collaborative_Recommendation/Keras/asset_encode2asset.pkl","rb"))

## Function Definition

In [ ]:
def get_recommendation(user_id,n):
            
    asset_read_by_user = df_keras_ratings[df_keras_ratings.uid == user_id]

    asset_not_read = asset_df[
        ~asset_df["asset_id"].isin(asset_read_by_user.asset_id.values)
    ]["asset_id"]

    asset_not_read = list(
        set(asset_not_read).intersection(set(asset2asset_encoded.keys()))
    )

    asset_not_read = [[asset2asset_encoded.get(x)] for x in asset_not_read]

    user_encoder = user2user_encoded.get(user_id)

    user_asset_array = np.hstack(
        ([[user_encoder]] * len(asset_not_read), asset_not_read)
    )


    ratings = model.predict(user_asset_array).flatten()

    top_ratings_indices = ratings.argsort()[-n:][::-1]

    recommended_asset_ids = [
        asset_encoded2asset.get(asset_not_read[x][0]) for x in top_ratings_indices
    ]

    print("Showing recommendations for user: {}".format(user_id))
    print("====" * 9)
    print("Assets with high ratings from user")
    print("----" * 8)
    top_assets_user = (
        asset_read_by_user.sort_values(by="rating", ascending=False)
        .head()
        .asset_id.values
    )
    asset_df_rows = asset_df[asset_df["asset_id"].isin(top_assets_user)]
    for row in asset_df_rows.itertuples():
        print(row.asset_id, ":", row.asset_title, ":",row.Domain)

    print("----" * 8)
    print("Top 5 article recommendations")
    print("----" * 8)

    for id in recommended_asset_ids:
        recommended_assets = asset_df[asset_df["asset_id"].isin([id])]
        for row in recommended_assets.itertuples():
            print(row.asset_id, ":", row.asset_title, ":",row.Domain)


## Get Recommendation

Users      Domain
*   1 - 12 :CX
*   13 - 24 : Agile
*   25 - 36 : AI
*   37 - 48 : IOT
*   49 - 60 : BigData
*   61 - 72 : Cloud
*   73 - 84 : HR
*   85 - 100 : Industry

In [ ]:
get_recommendation(1,5)

Showing recommendations for user: 1
Assets with high ratings from user
--------------------------------
17 : personalization boundaries tuning in to customers without turning them off : CX
18 : designing for the small silver screen : CX
20 : reimagining the contact center with digital technologies : CX
71 : how blockchain is poised to revolutionize digital customer experience : CX
76 : how quality engineering enables customer experience transformations : CX
--------------------------------
Top 5 article recommendations
--------------------------------
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
83 : tcs cmo study stage conversion personalizing content to turn prospects into customers : Industry
79 : tcs cmo study stage awareness attracting the digitally distracted prospect : Industry
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
74 : proactive protection how companies can secure customer data

In [ ]:
get_recommendation(14, 5)

Showing recommendations for user: 14
Assets with high ratings from user
--------------------------------
31 : how to make location independent agile work : Agile
33 : enterprise agility pushing innovation to the edge of the organization : Agile
35 : how retail ceos can drive agile to grow their business : Agile
36 : fending off the fintechs how agile financial services firms are transforming their businesses : Agile
43 : why agile software development requires radical changes in budgeting and scoping : Agile
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : Big

In [ ]:
get_recommendation(28, 5)

Showing recommendations for user: 28
Assets with high ratings from user
--------------------------------
3 : riding the cognitive wave : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
38 : where to turn ai and automation loose in your company : AI
42 : what happens when you turn your products into services : AI
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
8 : banking and financial services analytics for competitive advantage : BigData
1 : the ascent of the smarter faster lighter enterprise : CX
20 : reimagining the contact center with digital technologies : CX


In [ ]:
get_recommendation(47,5)

Showing recommendations for user: 47
Assets with high ratings from user
--------------------------------
19 : how iot technologies are revamping the post sale experience : IOT
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
44 : raising your iot security game : IOT
50 : using iot technologies to completely rethink the customer experience interview with ric merrifield : IOT
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : BigData


In [ ]:
get_recommendation(55,5)

Showing recommendations for user: 55
Assets with high ratings from user
--------------------------------
8 : banking and financial services analytics for competitive advantage : BigData
47 : analytics a still emerging business frontier interview with prabhudev konana : BigData
61 : operating in an insight driven world mastering the new game of analytics : BigData
--------------------------------
Top 5 article recommendations
--------------------------------
71 : how blockchain is poised to revolutionize digital customer experience : CX
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry


In [ ]:
get_recommendation(63,5)

Showing recommendations for user: 63
Assets with high ratings from user
--------------------------------
27 : cloudera the mind of the new machine : Cloud
40 : now you can simulate nearly anything : Cloud
60 : immense digital power on demand shifting to the brave new world of cloud computing : Cloud
72 : for companies today the question is not whether to move to the cloud but how : Cloud
73 : how the cloud empowers digital transformation : Cloud
--------------------------------
Top 5 article recommendations
--------------------------------
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry
56 : how the digital renovators are racing ahead : Industry


In [ ]:
get_recommendation(77,5)

Showing recommendations for user: 77
Assets with high ratings from user
--------------------------------
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
29 : students for life why executives must continually update their wisdom : HR
45 : executive traits for recognizing the bountiful opportunities ahead : HR
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
--------------------------------
Top 5 article recommendations
--------------------------------
55 : digital data s greatest opportunities : BigData
41 : the magic that happens when manufacturers digitally attach themselves to their products : IOT
59 : taking a machine first approach to digital transformation : AI
67 : unleashing business value in a business world : Industry
56 : how the digital renovators are racing ahead : Industry


In [ ]:
get_recommendation(80,5)

Showing recommendations for user: 80
Assets with high ratings from user
--------------------------------
11 : winning today s talent race how to identify develop and retain talent better than your rivals : HR
45 : executive traits for recognizing the bountiful opportunities ahead : HR
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
--------------------------------
Top 5 article recommendations
--------------------------------
18 : designing for the small silver screen : CX
13 : how microsoft uses machine learning to handle workloads that humans cannot : AI
7 : case study why artificial intelligence is a big part of ap s future : AI
8 : banking and financial services analytics for competitive advantage : BigData
1 : the ascent of the smarter faster lighter enterprise : CX


In [ ]:
get_recommendation(95,5)

Showing recommendations for user: 95
Assets with high ratings from user
--------------------------------
28 : manufacturing how to lighten up business models : Industry
32 : the view inside technologies collide and industries transform : Industry
46 : changing culture in a time of accelerating market change interview with vijay govindarajan : Industry
51 : increasing speed in the digital era interview with jeanne ross : Industry
81 : tcs cmo study how travel transportation and hospitality companies are digitally personalizing content to create market awareness : Industry
--------------------------------
Top 5 article recommendations
--------------------------------
84 : employee wellness analysis creating data driven hr policies that help save lives : HR
83 : tcs cmo study stage conversion personalizing content to turn prospects into customers : Industry
85 : why the emotional wellbeing of your employees should be a top priority during covid : HR
76 : how quality engineering enables cu